---
title: Decision Tree Induction with scikit-learn
math: 
    '\abs': '\left\lvert #1 \right\rvert' 
    '\norm': '\left\lvert #1 \right\rvert' 
    '\Set': '\left\{ #1 \right\}'
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
    '\Gini': '\operatorname{Gini}'
    '\Info': '\operatorname{Info}'
    '\Gain': '\operatorname{Gain}'
    '\GainRatio': '\operatorname{GainRatio}'
---

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, tree

%matplotlib widget
if not os.getenv(
    "NBGRADER_EXECUTION"
):
    %reload_ext jupyter_ai

## Decision Tree Induction

In this notebook, we will use scikit-learn to build decision trees on the [*iris dataset*](https://en.wikipedia.org/wiki/Iris_flower_data_set) from [`sklearn.datasets`](https://scikit-learn.org/stable/api/sklearn.datasets.html#module-sklearn.datasets) package.

In [ ]:
iris = datasets.load_iris()

Recall the classification task is to train a model that can automatically classify the species (*target*) based on the lengths and widths of the petals and sepals (*input features*).

To build a decision tree, we can use `DecisionTreeClassifier` from `sklearn.tree` and apply its `fit` method on the training set.

In [ ]:
clf_gini = tree.DecisionTreeClassifier(random_state=0).fit(iris.data, iris.target)

To display the decision tree, we can use the function `plot_tree` from `sklearn.tree`:

In [ ]:
plt.close(1)
plt.figure(num=1, clear=True)
tree.plot_tree(clf_gini)
plt.show()

::::{tip} If the figure is too small...

Drag the arrow at the bottom right of the figure to resize the bounding box. 

::::

For each node:
- `___ <= ___` is the splitting criterion for internal nodes, satisfied only by samples going left.
- `gini = ...` shows the impurity index. By default, the algorithm uses the Gini impurity index to find the best binary split. Observe that the index decreases down the tree towards the leaves.
- `value = [_, _, _]` shows the number of associated instances for each of the three classes.

::::{tip} How is the tree stored?

The information of the decision is stored in the `tree_` attribute of the classifier. For more details, run:

```python
help(clf_gini.tree_)
```

::::

Additional options may be provided to customize the look of the decision tree:

In [ ]:
options = {
    "feature_names": iris.feature_names,
    "class_names": iris.target_names,
    "label": "root",
    "filled": True,
    "node_ids": True,
    "proportion": True,
    "rounded": True,
    "fontsize": 7,
}  # store options as dict for reuse
plt.close(2)
plt.figure(num=2, clear=True, figsize=(9, 9))
tree.plot_tree(clf_gini, **options)  # ** unpacks dict as keyword arguments
plt.show()

Each node now indicates the majority class, which may be used as the decision. The majority classes are also color coded. Observe that the color gets lighter towards the root as the class distribution becomes more impure. In particular, the iris setosa is distinguished immediately after checking the petal width/length.

::::{exercise}
:label: ex:clf_entropy

Assign to `clf_entropy` the decision tree classifier created using *entropy* as the impurity measure. You can do so with the keyword argument `criterion='entropy'` in `DecisionTreeClassifier`. Furthermore, Use `random_state=0` and fit the classifier on the entire iris dataset. Check whether the resulting decision tree is the same as the one created using the Gini impurity index.

::::

In [ ]:
# YOUR CODE HERE
raise NotImplementedError
plt.close(3)
plt.figure(num=3, clear=True, figsize=(9, 9))
tree.plot_tree(clf_entropy, **options)
plt.show()

YOUR ANSWER HERE

::::{caution}

Although one can specify whether to use Gini impurity or entropy, `sklearn` implements neither [C4.5](https://en.wikipedia.org/wiki/C4.5_algorithm) nor [Classification and Regression Tree (CART)](https://en.wikipedia.org/wiki/Decision_tree_learning#Decision_tree_types). It supports only binary splits on numeric input attributes. See a workaround [here][categorical].

[categorical]: https://stackoverflow.com/questions/38108832/passing-categorical-data-to-sklearn-decision-tree

::::

In [ ]:
%%ai
Explain in a paragraph what one-hot encoding is, and why it is suitable for
converting categorical features to numeric values for scikit-learn decision trees,
which do not support categorical data?

## Splitting Criterion

To induce a good decision tree efficiently, the splitting criterion is chosen 
- greedily to maximize the reduction in impurity and 
- recursively starting from the root.

### Overview using pandas

To have a rough idea of what are good features to split on, we will use [pandas](https://pandas.pydata.org/docs/user_guide/index.html) [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame) 
to operate on the iris dataset.

In [ ]:
# write the input features first
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# append the target values to the last column
df["target"] = iris.target
df.target = df.target.astype("category").cat.rename_categories(
    dict(zip(range(3), iris.target_names))
)
df

To display some statistics of the input features for different classes:

In [ ]:
import warnings

plt.close(4)
fig = plt.figure(num=4, clear=True, figsize=(9, 9))
axs = fig.subplots(1, 3, sharex=False, sharey=True)

# Suppress that specific UserWarning due to an issue of Pandas
# https://stackoverflow.com/questions/55180754/pandas-scatter-matrix
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    df.groupby("target", observed=False).boxplot(ax=axs, rot=45, fontsize=8)

plt.subplots_adjust(bottom=0.25)
plt.tight_layout()
plt.show()

In [ ]:
df.groupby("target", observed=False).agg(["mean", "std"]).round(2)

::::{exercise}
:label: ex:good_features

Identify good feature(s) based on the above statistics. Does your choice agree with the decision tree generated by `DecisionTreeClassifier`?

::::

YOUR ANSWER HERE

### Measuring impurity

Suppose nearly all instances of a dataset belong to the same class. In that case, we can return the majority class as the decision without further splitting. A measure of impurity is the Gini impurity index, defined as follows:

::::{prf:definition} Gini impurity index
:label: def:gini

Given a dataset $D$ with a class attribute (discrete target), the Gini impurity index is defined as

$$
\Gini(D):= g(p_0,p_1,\dots)
$$ (Gini)

where $(p_0,p_1,\dots)$ are probability masses corresponding to the empirical class distribution of $D$, and

$$
\begin{align}
g(p_0,p_1,\dots) &:= \sum_k p_k(1-p_k)\\
&= 1- \sum_k p_k^2.
\end{align}
$$ (g)

::::

::::{note} For convenience, we may also write ...

- $g(\boldsymbol{p})$ for the stochastic vector $\boldsymbol{p}=\begin{bmatrix}p_0 & p_1 & \dots\end{bmatrix}$ of probability masses, and
- $g(p)$ for the probability mass function $p: k \mapsto p_k$.

::::

We can represent a distribution simply as a NumPy array. To return the empirical class distributions of the iris dataset:

In [ ]:
def dist(values):
    """
    Compute the empirical distribution of the given 1D array of values.

    This function calculates the empirical distribution of the input array,
    returning a 1D array of probabilities. The order of the probabilities in
    the output array is immaterial.

    Parameters
    ----------
    values : array-like
        A 1D array of values for which the empirical distribution is to be 
        computed.

    Returns
    -------
    probabilities : ndarray
        A 1D array of probabilities corresponding to the empirical distribution
        of the input values.

    Examples
    --------
    >>> import numpy as np
    >>> values = np.array([1, 2, 2, 3, 3, 3])
    >>> dist(values)
    array([0.16666667, 0.33333333, 0.5       ])
    """
    counts = np.unique(values, return_counts=True)[-1]
    return counts / counts.sum()


print(f"Distribution of target: {dist(iris.target).round(3)}")

The Gini impurity index can be implemented as follows:

In [ ]:
def g(p):
    """Returns the Gini impurity of the distribution p."""
    return 1 - (p**2).sum()
_code = In[-1].rsplit(maxsplit=1)[0] # store the code for chatting with LLM

In [ ]:
g?
print(f"Gini(D) = {g(dist(iris.target)):.3g}")

In [ ]:
%%ai
Improve the docstring to conform to NumPy style:
--
{_code}

::::{seealso} How to document your code?

Properly documenting code ensures that others can reuse the code and 
collaborate to improve it. The NumPy style is a popular format for writing docstrings, which are string literals included in the code to document their usage. 

A more comprehensive documentation can be created using Sphinx, which includes the autodoc and Napolean extensions that can automatically extract, parse, and generate documentation from docstrings in NumPy style. To try it out, see the [documentation of Sphinx](https://www.sphinx-doc.org/en/master/usage/extensions/napoleon.html).

::::

Another measure of impurity uses the information measure called entropy in information theory:

::::{prf:definition} entropy
:label: def:entropy

The information content is defined as 

$$
\Info(D):= h(p_0,p_1,\dots),
$$ (Info)

which is the *entropy* of the class distribution

$$
\begin{align}
h(\boldsymbol{p}) = h(p_0,p_1,\dots) &= \sum_{k:p_k>0} p_k \log_2 \frac1{p_k}.
\end{align}
$$ (h)

::::

::::{prf:remark}

- For convenience, we often omit the constraint $p_k>0$ by regarding $0 \log \frac10$ as the limit $\lim_{p\to 0} p \log \frac1{p} = 0$.
- Unless otherwise specified, all the logarithm is base 2, i.e., $\log = \lg$, where the information quantities are in the unit of bit (binary digit). A popular alternative is to use the natural logarithm, $\log = \ln$, where the unit is in *nat*.

::::

::::{exercise}
:label: ex:h

Complete the following function to compute the entropy of a distribution. You may use the function `log2` from `numpy` to calculate the logarithm base 2.

:::{hint}
:class: dropdown

Consider the solution template:

```python
def h(p):
    ...
    return (p * ___ * ___).sum()
```

:::


::::

In [ ]:
def h(p):
    # Improve the docstring below to conform to NumPy style.
    """Returns the entropy of distribution p (1D array)."""
    p = np.array(p)
    p = p[(p > 0) & (p < 1)]  # 0 log 0 = 1 log 1 = 0
    # YOUR CODE HERE
    raise NotImplementedError


h?
print(f"Info(D): {h(dist(iris.target)):.3g}")

In [ ]:
# tests
assert np.isclose(h([1 / 2, 1 / 2]), 1)
assert np.isclose(h([1, 0]), 0)
assert np.isclose(h([1 / 2, 1 / 4, 1 / 4]), 1.5)

In [ ]:
# hidden tests

### Drop in impurity

The CART algorithm uses the drop in Gini impurity to select splitting attributes:

::::{prf:definition} 

The *drop in Gini impurity* for a splitting criterion $A$ on a dataset $D$ with respect to the class attribute is defined as

$$
\begin{align}
\Delta \Gini_{A}(D) &:= \Gini(D) - \Gini_{A}(D)\\
\Gini_{A}(D) &:= \sum_{j} \frac{|D_j|}{|D|} \Gini(D_j),
\end{align}
$$ (Delta-Gini)

where $D$ is split by $A$ into $D_j$ for different outcomes $j$ of the split.

::::

We will consider the binary splitting criterion $\R{X}\leq s$ in particular, which gives

$$
\begin{align}
\Delta \Gini_{A}(D) = g(\hat{P}_\R{Y}) - \left[\hat{P}[\R{X}\leq s] g(\hat{p}_{\R{Y}|\R{X}\leq s}) + \hat{P}[\R{X}> s]g(\hat{p}_{\R{Y}|\R{X}> s})\right]
\end{align}
$$ (Delta-Gini-binary)

where 

- $\R{Y}$ denotes the target,
- $\hat{P}$ denotes the empirical distribution, and
- $\hat{p}_{\R{Y}|\R{X}\leq s}$, $\hat{p}_{\R{Y}|\R{X}> s}$, and $\hat{p}_{\R{Y}}$ denote the empirical probability mass functions of $\R{Y}$ with or without conditioning.

In [ ]:
def drop_in_gini(X, Y, s):
    """
    Calculate the drop in Gini impurity for a binary split.

    This function computes the reduction in Gini impurity of the target `Y`
    when the input feature `X` is split at the threshold `s`.

    Parameters
    ----------
    X : 1D array-like
        Input feature values for different instances.
    Y : 1D array-like
        Target values corresponding to `X`.
    s : float
        Splitting point for `X`.

    Returns
    -------
    float
        The reduction in Gini impurity resulting from the split.
    """
    S = X <= s
    q = S.mean()
    return g(dist(Y)) - q * g(dist(Y[S])) - (1 - q) * g(dist(Y[~S]))


X, Y = df["petal width (cm)"], df.target
print(f"Drop in Gini: {drop_in_gini(X, Y, 0.8):.4g}")

To compute the best splitting point for a given input feature, we check every consecutive mid-points of the observed feature values:

In [ ]:
def find_best_split_pt(X, Y, gain_function):
    """
    Find the best splitting point and the maximum gain for a binary split.

    This function identifies the optimal splitting point `s` that maximizes the
    gain as evaluated by the provided `gain_function` for the split `X <= s` 
    and target `Y`.

    Parameters
    ----------
    X : 1D array-like
        Input feature values for different instances.
    Y : 1D array-like
        Target values corresponding to `X`.
    gain_function : function
        A function that evaluates the gain for a splitting criterion `X <= s`.
        For example, `drop_in_gini`.

    Returns
    -------
    tuple
        (s, g) where `s` is the best splitting point and `g` is the maximum 
        gain.

    See Also
    --------
    drop_in_gini : Function to calculate the drop in Gini impurity.
    """
    values = np.sort(np.unique(X))
    split_pts = (values[1:] + values[:-1]) / 2
    gain = np.array([gain_function(X, Y, s) for s in split_pts])
    i = np.argmax(gain)
    return split_pts[i], gain[i]


print(
    """Best split point: {0:.3g}
Maximum gain: {1:.3g}""".format(
        *find_best_split_pt(X, Y, drop_in_gini)
    )
)

The following ranks the features according to the gains of their best binary splits:

In [ ]:
rank_by_gini = pd.DataFrame(
    {
        "feature": feature,
        **(lambda s, g: {"split point": s, "gain": g})(
            *find_best_split_pt(df[feature], df.target, drop_in_gini)
        ),
    }
    for feature in iris.feature_names
).sort_values(by="gain", ascending=False)
rank_by_gini

Using the entropy to measure impurity, we have the following alternative gain function:

::::{prf:definition} information gain

The information gain is defined as 

$$
\begin{align}
\Gain_{A}(D) &:= \Info(D) - \Info_{A}(D) && \text{where}\\
\Info_{A}(D) &:= \sum_{j} \frac{|D_j|}{|D|} \Info(D_j),
\end{align}
$$ (Gain)

::::

We will again consider the binary splitting criterion $\R{X}\leq s$ in particular, which gives

$$
\begin{align}
\Gain_{\R{X}\leq s}(D) = h(\hat{P}_Y) - \left[\hat{P}[\R{X}\leq s] h(\hat{p}_{\R{Y}|\R{X}\leq s}) + \hat{P}[\R{X}> s]h(\hat{p}_{\R{Y}|\R{X}> s})\right]
\end{align}
$$ (Gain-binary)

::::{exercise}
:label: ex:gain

Complete the following function to calculate the information gain on the target $\R{Y}$ for a binary split $\R{X}\leq s$. You may use `dist` and `h` defined previously.

::::

In [ ]:
def gain(X, Y, s):
    """
    Calculate the information gain for a binary split.

    This function computes the information gain of the target `Y` when the 
    input feature `X` is split at the threshold `s`.

    Parameters
    ----------
    X : 1D array-like
        Input feature values for different instances.
    Y : 1D array-like
        Target values corresponding to `X`.
    s : float
        Splitting point for `X`.

    Returns
    -------
    float
        The information gain resulting from the split.
    """
    S = X <= s
    q = S.mean()
    # YOUR CODE HERE
    raise NotImplementedError


print(f"Information gain: {gain(X, Y, 0.8):.4g}")

In [ ]:
# tests
rank_by_entropy = pd.DataFrame(
    {
        "feature": feature,
        **(lambda s, g: {"split point": s, "gain": g})(
            *find_best_split_pt(df[feature], df.target, gain)
        ),
    }
    for feature in iris.feature_names
).sort_values(by="gain", ascending=False)
rank_by_entropy

In [ ]:
# hidden tests

The C4.5 tree induction algorithm uses information gain ratio instead of information gain:

::::{prf:definition} gain ratio

The *information gain ratio* is defined as 

$$
\begin{align}
\GainRatio_{A}(D) &:= \frac{\Gain_{A}(D)}{\operatorname{SplitInfo}_{A}(D)}
\end{align}
$$ (GainRatio)

which is normalized by 

$$
\begin{align}
\operatorname{SplitInfo}_{A}(D) &:= h\left(j\mapsto \frac{|D_j|}{|D|} \right)=\sum_j \frac{|D_j|}{|D|}\log \frac{|D|}{|D_j|}.
\end{align}
$$ (SplitInfo)

::::

For binary split $\R{X}\leq s$,

$$
\operatorname{SplitInfo}_{\R{X}\leq s}(D) := h\left(\hat{P}[\R{X}\leq s], \hat{P}[\R{X}> s]\right)
$$ (SplitInfo-binary)

in terms of the empirical distribution.

::::{exercise}
:label: ex:gain_ratio

Complete the following function to calculate the *information gain ratio* for a binary split $\R{X}\leq s$ and target $\R{Y}$.

::::

In [ ]:
def gain_ratio(X, Y, split_pt): 
    # Add docstring here
    S = X <= split_pt
    q = S.mean()
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
rank_by_gain_ratio = pd.DataFrame(
    {
        "feature": feature,
        **(lambda s, g: {"split point": s, "gain": g})(
            *find_best_split_pt(df[feature], df.target, gain_ratio)
        ),
    }
    for feature in iris.feature_names
).sort_values(by="gain", ascending=False)
rank_by_gain_ratio

In [ ]:
# hidden tests

::::{exercise}
:label: ex:difference

Does the information gain ratio give a different ranking of the features in the above? Why?

::::

YOUR ANSWER HERE